In [1]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np

# Initialize the camera and MediaPipe Face Mesh
cam = cv2.VideoCapture(0)
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)
screen_w, screen_h = pyautogui.size()

while True:
    # Capture frame from webcam
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Flip the frame horizontally (mirror effect)
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB for MediaPipe (ensure it's numpy ndarray)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Ensure the type is correct and is numpy array
    rgb_frame = np.array(rgb_frame)

    try:
        # Process the frame to get the face landmarks
        output = face_mesh.process(rgb_frame)

        # Check if any landmarks were detected
        if output.multi_face_landmarks:
            landmark_points = output.multi_face_landmarks[0].landmark  # Get the first face's landmarks

            frame_h, frame_w, _ = frame.shape

            # Draw and track the eye region landmarks (landmarks 474-478)
            for id, landmark in enumerate(landmark_points[474:478]):
                x = int(landmark.x * frame_w)
                y = int(landmark.y * frame_h)
                cv2.circle(frame, (x, y), 3, (0, 255, 0))

                if id == 1:
                    # Move the cursor based on the eye position
                    screen_x = screen_w * landmark.x
                    screen_y = screen_h * landmark.y
                    pyautogui.moveTo(screen_x, screen_y)

            # Detect if the left eye is blinking (landmarks 145 and 159)
            left = [landmark_points[145], landmark_points[159]]
            for landmark in left:
                x = int(landmark.x * frame_w)
                y = int(landmark.y * frame_h)
                cv2.circle(frame, (x, y), 3, (0, 255, 255))

            # If the eye is closed (small vertical distance between eyelid points), click
            if (left[0].y - left[1].y) < 0.004:
                pyautogui.click()
                pyautogui.sleep(1)

    except Exception as e:
        print(f"Error during processing: {e}")

    # Display the frame with the drawn landmarks
    cv2.imshow('Eye Controlled Mouse', frame)

    # Break the loop if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cam.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 